## <center>卷积神经网络</center>

### 1 理解卷积神经网络

卷积神经网络,使用权重共享的方式构建网络,避免了对参数的过度依赖,相比于全连接网络,能更好的识别高维数据. 卷积神经网络的结构主要包括卷积层,池化层.一个卷积神经网络主要包括以下5个部分:

+ 输入层:将每个像素代表一个特征节点输入进来
+ 卷积操作:由多个滤波器组合的卷积层
+ 池化层:将卷积结果降维
+ 全局平均池化层:对生成的feature map 取平均值
+ 输出层:需要分成几个类,相应的就会有几个输出节点.每个输出节点都代表当前样本属于的该类的概率

<font color='red'>注意</font>

全局平均池化层是后出的新技术,以前这个位置通常使用 1-3 层全连接层来替代. 全连接层的劣势在于会产生大量的计算,需要大量的参数,但在效果上却和全局平均池化层一样.

### 2 卷积操作
卷积分为窄卷积,全卷积和同卷积
<center><img src="./img/6/1.png" width="600"></center>

#### (1) 步长

步长通常使用 stride 来表示, 上图中 filter 表示卷积核, 每个卷积核可以输出一个 feature map. stride =1 表示每次移动一步. 卷积操作就是,原图与filter 对应位置直接相乘,再将其结果相加.

#### (2)窄卷积

窄卷积(<font color='red'>valid卷积</font>),生成的feature map 比原始图片小,它的步长是可变的. 假如 滑动的步长为 $S$, 原始图片的维度为 $N_1 \times N_1$, 并且卷积核的大小为 $N_2 \times N_2$ ,那么卷积之后的图像大小为 $floor(\frac{N_1 - N_2}{S}) + 1$

#### (3)同卷积

同卷积(<font color='red'>same卷积</font>),生成的feature map 与原始图片大小相同,它的步长是不变的,滑动步长都是1.一般操作时都需要使用padding技术(外围补一圈0,以确保生成的尺寸不变).

#### (4) 全卷积
全卷积(<font color="red">full 卷积</font>),也叫反卷积.就是把原始图片里的每个像素用卷积操作展开.如图,白色的表示原始的图片,灰色的表示卷积核,黑色的表示正在进行卷积操作的像素.
<center><img src="./img/6/2.png" width="600"/></center>

#### (5)多通道卷积
通道是指图片中每个像素由几个数来表示,例如黑白图片,通道数是1,彩色图片通道数是3,在卷积神经网络中,通道又分为输入通道和输出通道.
+ 输入通道:如果是彩色图片,那么起始输入通道就是3. 如果是中间层的卷积,输入通道就是上一层的输出通道. 计算方法是, <font color="red">每个输入通道</font>的图片都<font color="red">使用同一个卷积核</font>进行卷积操作,生成与输入通道匹配的 feature map (例如,彩色图片就是3个) ,然后再将这几个feature map 相同的位置熵的值加起来,生成一张feature map

+ 输出通道:想要输出几个feature map,就放几个卷积核,这就是输出通道

#### (6) 池化层
池化层的主要目的是降维,即在保持原有特征的基础上,最大限度地将数组的维数变小.池化步骤也有步长,和卷积非常像.
+ **均值池化**: 就是计算在卷积核对应的区域中,<font color='red'>所有不为0的像素点均值</font>,只能计算不为0的点,否则影响结果.
+ **最大池化**: 就是计算在卷积核对应的区域中,选择放最大值.

### 2 卷积神经网络中相关函数

#### (1) 卷积函数
Tensorflow中使用 tf.nn.conv2d 函数来实现卷积,其格式如下:
```python
tf.nn.conv2d(input, filter, strides, padding, use_cudnn_on_gpu=None)

input: 原图像,要求是一个Tensor,形状为 [batch,height,width,channels] 并且数据类型要求是tf.float32 或 tf.float64

filter: 卷积核,要求是一个Tensor,形状为 [height,width,in_channels,out_channels] 要求in_channels和input 的通道数量相同

strides: 卷积是每一维的步长,是一个一维的向量,长度为4. 需要结合input中的数据计算

padding: 定义元素边框与元素内容之间的空间. string 类型的变量,只能是 "SAME" 和 "VALID",这个值决定了不同的卷积方式,padding 的值为 "VALID" 时,表示边缘不填充. 当其填充为 "SAME"时, 表示填充到滤波器可以达到图像边缘.

use_cudnn_on_gpu: bool 类型,是否使用 cudnn 加速,默认为true.

返回值: tf.nn.conv2d 函数结果返回一个Tensor, 这个输出就是常说的 feature map
```

<font color='red'>注意</font>

在卷积函数中,padding 参数是最容易引起歧义的, 该参数仅仅决定是否要补0, 因此一定要清楚, padding 设置为 "SAME" 的真正含义,在设为 SAME 的情况下,只有在步长为 1 时,生成的feature map 才会与输入值相等.

#### (2) 池化函数